## Problema a ser solucionado

Uma empresa do setor audiovisual deseja aumentar a probabilidade de sucesso comercial de um novo filme. Para isso, disponibilizou um conjunto de dados históricos sobre produções anteriores e solicitou uma análise exploratória com o objetivo de identificar padrões, tendências e fatores que influenciam o desempenho de um filme.

Com base nessa análise, o objetivo é gerar insights acionáveis que orientem decisões estratégicas na produção do novo filme, como escolhas relacionadas a duração, gênero, classificação e demais características associadas a filmes bem-sucedidos.

### Carregamento e Tratamento dos Dados

In [419]:
import pandas as pd 
pd.set_option('display.max_columns', None)

df = pd.read_csv("../data/desafio_indicium_imdb.csv")
df.head(5)

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


Aqui nota-se que a coluna "Unnamed: 0" pode ser simplesmente descartada, pois nao tem nenhuma informação relevante! 

In [420]:
df = df.drop(columns=['Unnamed: 0'])

In [421]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   999 non-null    object 
 1   Released_Year  999 non-null    object 
 2   Certificate    898 non-null    object 
 3   Runtime        999 non-null    object 
 4   Genre          999 non-null    object 
 5   IMDB_Rating    999 non-null    float64
 6   Overview       999 non-null    object 
 7   Meta_score     842 non-null    float64
 8   Director       999 non-null    object 
 9   Star1          999 non-null    object 
 10  Star2          999 non-null    object 
 11  Star3          999 non-null    object 
 12  Star4          999 non-null    object 
 13  No_of_Votes    999 non-null    int64  
 14  Gross          830 non-null    object 
dtypes: float64(2), int64(1), object(12)
memory usage: 117.2+ KB


Aqui é encontrado mais alguns problemas, nossa base de dados possui valores nulos e as colunas "Released_Year", "Runtime" e "Gross" estão como Object, o que nao está correto, isso porque quero trabalhar com os dados no valor numerico, e dessa forma nao é possivel.

In [422]:
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')
df['Released_Year'] = df['Released_Year'].astype('Int64')

Nessa etapa eu tentei transformar os valores de 'Released_Year' diretamente para inteiro, porém não foi possivel, pois existe valor nao numerico nos dados, entao segui a seguinte abordagem:

- Transformei cada valor de para número, estes valores primeiramente foram transformados em float, e caso nao consiguisse por ter valores nao numericos, converteria para NaN. (depois tratarei esses casos).
- Após isso, passei todos eles para o tipo int, porem o Int64, que é o int do pandas que aceita valores nulos.

In [423]:
df['Runtime'] = df['Runtime'].str.replace('min', '').astype(int)

Aqui apenas substitui a palavra 'min' por nada, ao remover essa string dos dados de runtime foi possivel fazer a conversão

In [424]:
df['Gross'] = df['Gross'].str.replace(',', '').astype(float)

E retiro as ',' dos valores de 'Gross' para tambem poder fazer a convesão dos dados

In [425]:
df.isna().sum()

Series_Title       0
Released_Year      1
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

Outro problema, a base de dados nao é grande e possui muitos valores nulos, se e simplesmente remover tudo, será perdido um volume muito grande de dados. Para que eu possa amenizar essa perda sem distorcer muito o conjunto de dados, a a melhor forma é substituir os valores nulos por algo coerente, vou explorar isso...

##### Explorando e conhecendo mais os dados de "Gross", "Meta_score", "Certificate" e "Released_Year" para tratamento

In [426]:
df['Gross'].describe().round(1)

count          830.0
mean      68082574.1
std      109807553.4
min           1305.0
25%        3245338.5
50%       23457439.5
75%       80876340.2
max      936662225.0
Name: Gross, dtype: float64

In [429]:
df.groupby(['IMDB_Rating'])['Gross'].describe().round(1)

,count,mean,std,min,25%,50%,75%,max
IMDB_Rating,,,,,,,,
7.6,109.0,70104547.6,95337194.8,6013.0,10301706.0,39868642.0,88513495.0,608581744.0
7.7,136.0,58849937.5,77364263.3,26331.0,4771792.2,27815753.5,76805035.8,324591735.0
7.8,122.0,75684709.5,127491859.2,1305.0,4010172.8,32449205.5,90463203.5,760507625.0
7.9,89.0,52405019.8,118231818.7,61001.0,3237118.0,12281500.0,44671682.0,936662225.0
8.0,113.0,71311990.9,102614442.1,3296.0,2625803.0,24475416.0,102308889.0,623279547.0
8.1,99.0,52492028.7,92921469.2,14131.0,1073507.0,5820649.0,57886155.5,402453882.0
8.2,51.0,45537860.6,82605670.1,7461.0,937975.0,5014000.0,59565340.0,415004880.0
8.3,39.0,47587745.6,68020898.2,28877.0,3417741.0,15000000.0,54464010.5,309125409.0
8.4,25.0,133760928.2,222398701.8,275902.0,5017246.0,25544867.0,190241310.0,858373000.0


Como a variável Gross apresenta grande variaçao entre seus valores minimo e maximo, alem de uma media significativamente maior que a mediana, optei pela imputaçao dos valores ausentes utilizando a mediana, por ser mais robusta a outliers e representar melhor o faturamento normal de um filme. Para tornar essa imputaçao mais contextualizada, os filmes foram agrupados pela nota do IMDb, substituindo os valores nulos pela mediana do grupo correspondente, assumindo que filmes com avaliações semelhantes tendem a ter desempenhos financeiros comparaveis.

In [ ]:
df['Gross'] = df.groupby('IMDB_Rating')['Gross'].transform(lambda x: x.fillna(x.median()))

# Continua daqui!

In [ ]:
df['Meta_score'].describe().round(1)

count    842.0
mean      78.0
std       12.4
min       28.0
25%       70.0
50%       79.0
75%       87.0
max      100.0
Name: Meta_score, dtype: float64

Aqui notamos que nao tem uma diferença tao grande em relação a média e a mediana, optarei por utilizar a média!

In [ ]:
df['Meta_score'] = df['Meta_score'].fillna(df['Meta_score'].mean())

Esse caso foi simples, apenas pesquisei na internet e vi que o filme foi lançado em 1995

In [ ]:
df[df['Released_Year'].isna()]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
965,Apollo 13,<NA>,U,140,"Adventure, Drama, History",7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,Tom Hanks,Bill Paxton,Kevin Bacon,Gary Sinise,269197,173837933.0


In [ ]:
df['Certificate'].value_counts(dropna=False)

Certificate
U           234
A           196
UA          175
R           146
NaN         101
PG-13        43
PG           37
Passed       34
G            12
Approved     11
TV-PG         3
GP            2
TV-14         1
Unrated       1
TV-MA         1
16            1
U/A           1
Name: count, dtype: int64

Aqui eu optarei apenas por retirar os Dados nulos, tenho receio desse caso eu adicionar algum valor e alterar o resultado das analises!

In [ ]:
df['Released_Year'] = df['Released_Year'].fillna(1995)

In [ ]:
df.isna().sum()

Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score         0
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross              0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(898, 15)

Dessa forma conseguimos limpar os dados tendo o menor numero de perdas e distorção dos dados, isso vai nos garantir uma analise mais consistente mais para frente!

In [ ]:
df.duplicated(subset=["Series_Title", "Released_Year"]).sum()

np.int64(0)

In [ ]:
df.head(5)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
1,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
2,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0
3,12 Angry Men,1957,U,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0
4,The Lord of the Rings: The Return of the King,2003,U,201,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,377845905.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 898 entries, 0 to 996
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   898 non-null    object 
 1   Released_Year  898 non-null    Int64  
 2   Certificate    898 non-null    object 
 3   Runtime        898 non-null    int64  
 4   Genre          898 non-null    object 
 5   IMDB_Rating    898 non-null    float64
 6   Overview       898 non-null    object 
 7   Meta_score     898 non-null    float64
 8   Director       898 non-null    object 
 9   Star1          898 non-null    object 
 10  Star2          898 non-null    object 
 11  Star3          898 non-null    object 
 12  Star4          898 non-null    object 
 13  No_of_Votes    898 non-null    int64  
 14  Gross          898 non-null    float64
dtypes: Int64(1), float64(3), int64(2), object(9)
memory usage: 113.1+ KB


Os dados agora estão nos formatos corretos e totalmente preenchidos ...<br>
Agora criarei um dicionario para o "Certificate", porque pelo que vi essas classificaçoes, tem algumas que sao a mesma coisa porem para países diferentes

In [ ]:
print("temos: ", df["Certificate"].value_counts())

mapa_etaria = {
    'U': 0,
    'G': 0,
    'Approved': 0,
    'Passed': 0,
    'GP': 0,
    'UA': 1,
    'U/A': 1,
    'PG': 1,
    'PG-13': 1,
    'TV-PG': 1,
    'A': 2,
    'R': 2,
    'TV-14': 2,
    '16': 2,
    'TV-MA': 2,
    'Unrated': 2
}

df['Faixa_etaria'] = df['Certificate'].map(mapa_etaria)
df['Faixa_etaria_texto'] = df['Faixa_etaria'].map({0:'Livre', 1:'Adolescentes', 2:'Adulto'}) 

temos:  Certificate
U           234
A           196
UA          175
R           146
PG-13        43
PG           37
Passed       34
G            12
Approved     11
TV-PG         3
GP            2
TV-14         1
Unrated       1
TV-MA         1
16            1
U/A           1
Name: count, dtype: int64


In [ ]:
df.head(5)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Faixa_etaria,Faixa_etaria_texto
0,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0,2,Adulto
1,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0,1,Adolescentes
2,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0,2,Adulto
3,12 Angry Men,1957,U,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0,0,Livre
4,The Lord of the Rings: The Return of the King,2003,U,201,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,377845905.0,0,Livre


In [ ]:
df.to_csv("../data/cinema_processed.csv", index=False)